<a href="https://colab.research.google.com/github/YoshiyukiKono/langchain_for_beginners/blob/main/02_RAG_table_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain DBテーブルのテキストデータを使ったRAG

https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.cassandra.Cassandra.html#

## パッケージインストール

In [ ]:
!pip install --upgrade astrapy==0.7.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.8/18.8 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00


In [2]:
!pip install openai==1.8.0 tiktoken==0.5.2 cohere==4.44
!pip install langchain==0.1.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.3/222.3 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
  Attempting uninstall: importlib_metadata
    Found existing installation: importlib-metadata 7.0.1
    Uninstalling importlib-metadata-7.0.1:
      Successfully uninstalled importlib-metadata-7.0.1
ERROR: pip's dependency resolver does not cu

## Astra DB 接続確認

【下記セルを実行すると入力欄が表示されます】

`AstraCS:`から始まるトークンを設定します。


In [ ]:
import getpass

YOUR_TOKEN = getpass.getpass()


··········


【下記セルを実行すると入力欄が表示されます】

あなたのAstra環境に表示されているサンプルプログラムからAPIエンドポイントの値をコピーして、設定します。


In [ ]:

API_ENDPOINT = input()

https://95154b76-8356-4fb2-89f3-57779b833dfb-us-east1.apps.astra.datastax.com


In [ ]:
from astrapy.db import AstraDB

# Initialization
db = AstraDB(
  token=YOUR_TOKEN,
  api_endpoint=API_ENDPOINT)

print(f"Connected to Astra DB: {db.get_collections()}")

Connected to Astra DB: {'status': {'collections': []}}


必要に応じ既存のコレクションを削除します。

In [ ]:
db.delete_collection(collection_name="vector_test")

{'status': {'ok': 1}}

In [ ]:
db.delete_collection(collection_name="book_collection")

{'status': {'ok': 1}}

### DATA

In [ ]:
%%writefile books.csv
ID,title,author,publisher,price,year,description
001,ネットワークセキュリティ入門,山田太郎,技術書典,4500,2021,ネットワークセキュリティの基本原則と実践的な手法に焦点を当てた入門書。
002,ビジネスデータ分析の基礎,田中花子,ビジネスブックス,3800,2020,ビジネスにおけるデータ分析の基本的な手法とツールの使い方を解説。
003,クラウドアーキテクチャデザイン,鈴木次郎,技術出版株式会社,5500,2022,クラウドネイティブなアプリケーションのアーキテクチャデザインとベストプラクティスを詳細に解説。
004,金融取引のアルゴリズム,林美香,データ出版社,5200,2019,金融市場におけるアルゴリズムトレーディングの基本原則と実践手法に焦点を当てた実践的なガイド。
005,人事マネジメントの実践,佐藤健太,ビジネスパートナーズ,4800,2020,組織内の人材マネジメントに関する基本的な原則と実践的な手法を紹介。
006,自然言語処理入門,伊藤さやか,技術書典,4200,2021,自然言語処理の基本概念から、実践的なテキストデータ処理手法までを解説。
007,マーケティング戦略の立案,山口直樹,ビジネスブックス,3600,2018,市場分析、セグメンテーション、ポジショニングなど、マーケティング戦略の構築手法を学ぶ。
008,ディープラーニングアプリケーション,田村徹,技術出版株式会社,6500,2019,ディープラーニングを使用した実践的なアプリケーション開発手法に焦点を当てた実践書。
009,投資ポートフォリオ管理,高橋優一,金融情報出版,3700,2020,効果的な投資ポートフォリオの構築と管理に関する手法を解説。
010,サイバーセキュリティ実践ガイド,中村あや,技術書典,4800,2021,サイバーセキュリティの基本から、脆弱性診断、セキュリティ対策までを詳細に解説。
011,ビジネスリーダーシップの原則,大塚和夫,ビジネスパートナーズ,4200,2020,ビジネスにおけるリーダーシップの基本原則と実践的なリーダーシップスキルの向上を目指す手法を紹介。
012,コンピュータネットワークの基礎,西村さゆり,技術書典,3500,2022,コンピュータネットワークの基本概念から、実践的なネットワーク設計までを解説。
013,ビジネスプロセス最適化,小林太郎,ビジネスブックス,4600,2019,ビジネスプロセスの最適化手法とツールを駆使して企業の効率を向上させる手法を詳細に解説。
014,データウェアハウス設計入門,川上美香,データ出版社,5000,2021,データウェアハウスの基本概念から、実践的な設計手法までを解説。
015,マネジャーのためのコーチング,森田健太,ビジネスパートナーズ,3900,2018,リーダーシップとコーチングの基本原則と実践的なスキル向上のための手法を解説。
016,組み込みシステム開発ガイド,伊藤美和,技術出版株式会社,5500,2019,組み込みシステムの開発手法とデバッグテクニックに焦点を当てた実践的なガイド。
17,ディープラーニング実践入門,岡田淳,技術出版株式会社,4800,2022,ディープラーニングの基本理論から実践的な応用までを解説。
18,ビジネスプロセス改善ガイド,松本美香,ビジネスブックス,3500,2021,効果的なビジネスプロセス改善手法とツールを紹介。
19,Pythonデータサイエンス,田中健太,技術書典,4200,2020,Pythonを使用したデータサイエンスの基本と実践手法を学ぶ。
20,金融工学入門,林美香,データ出版社,5500,2019,金融商品の設計と価格付けに関する基本原則と数学的手法の解説。
21,チームリーダーシップの手引き,佐藤健太,ビジネスパートナーズ,4000,2020,効果的なチームリーダーシップの構築と維持に関するガイド。
22,React.js開発入門,伊藤さやか,技術書典,3800,2022,React.jsを使用したモダンなWebアプリケーションの開発手法を学ぶ。
23,戦略的マーケティング,山口直樹,ビジネスブックス,4500,2018,市場分析と戦略の構築に焦点を当てたマーケティング戦略のガイド。
24,機械学習プロジェクト管理,田村徹,技術出版株式会社,4900,2019,機械学習プロジェクトの計画と実行に関する手法を解説。
25,投資戦略の戦術,高橋優一,金融情報出版,3600,2021,リスク管理と効果的な投資戦略の構築に焦点を当てた投資戦略の戦術を解説。
26,サイバーセキュリティ対策,中村あや,技術書典,4300,2021,サイバーセキュリティの基本概念と対策手法を解説。
27,効果的なプロジェクトマネジメント,大塚和夫,ビジネスパートナーズ,4100,2020,プロジェクトの計画と実行に関する基本原則と実践的な手法を学ぶ。
28,データウェアハウス設計と実践,川上美香,データ出版社,5200,2021,データウェアハウスの設計と実践的なデータ統合手法を解説。
29,マーケットリサーチ入門,森田健太,ビジネスブックス,3700,2018,効果的なマーケットリサーチの実施とデータ分析手法を学ぶ。
30,クラウドセキュリティベストプラクティス,西村さゆり,技術出版株式会社,4700,2019,クラウド環境でのセキュリティのベストプラクティスを解説。
31,データサイエンティストのための統計学,小林太郎,データ書典,4400,2020,データサイエンスにおける統計学の基本と高度な手法までを解説。
32,リーダーシップ開発プログラム,中村美和,ビジネスパートナーズ,3900,2021,リーダーシップスキルの開発とチームビルディングの手法を学ぶ。
33,モバイルアプリケーション開発,高橋健一,技術書典,5000,2019,モバイルアプリケーションの開発手法とデザイン原則に焦点を当てた実践的なガイド。
34,投資ポートフォリオ戦略,田村美香,金融情報出版,4100,2020,リスク管理と効果的なポートフォリオの構築に焦点を当てた投資戦略の構築手法を解説。
35,サイバーセキュリティ実践入門,伊藤健太,技術書典,4600,2021,サイバーセキュリティの基本から、実践的な対策手法までを詳細に解説。
36,量子コンピューティング入門,岡本太郎,技術書典,5800,2022,量子コンピューティングの基礎理論から実践的な応用までを解説。
37,組織開発と変革マネジメント,村田花子,ビジネスブックス,4200,2021,組織の成熟度と変革マネジメントの手法に焦点を当てた実践的なガイド。
38,Javaマイクロサービスアーキテクチャ,佐々木健太,技術出版株式会社,4800,2020,Javaを使用したマイクロサービスアーキテクチャの基本と設計手法を学ぶ。
39,新時代のプロダクトマネジメント,林さやか,ビジネスパートナーズ,5500,2019,デジタルプロダクトのマネジメントにおける新たな手法とアプローチを解説。
40,ネットワークセキュリティ実践ガイド,高橋徹,技術書典,4600,2020,ネットワークセキュリティの実践的な対策手法とセキュアなネットワーク構築手法を学ぶ。
41,スタートアップのための財務戦略,伊東美和,ビジネスブックス,4000,2021,スタートアップ企業の財務戦略と資金調達の手法に焦点を当てたガイド。
42,自然言語処理と機械翻訳,山岸直樹,技術出版株式会社,5300,2018,自然言語処理と機械翻訳の基本原則と応用手法を学ぶ。
43,企業価値の最大化,佐野美香,データ出版社,4900,2019,企業価値の向上と最大化に向けた戦略と実践手法を解説。
44,データサイエンスとビジネス分析,鈴木健一,金融情報出版,3600,2020,データサイエンスとビジネスアナリティクスの基本概念と手法を学ぶ。
45,クラウドセキュリティポリシー,中川美和,技術書典,4400,2021,クラウド環境でのセキュリティポリシーの設計と実践手法に焦点を当てた実践ガイド。
46,リーダーシップ心理学,大西美香,ビジネスパートナーズ,4100,2018,リーダーシップと心理学の関連性と、組織内での実践手法を解説。
47,フロントエンド開発ベストプラクティス,田島健太,技術出版株式会社,4900,2019,モダンなフロントエンド開発のベストプラクティスと設計原則に焦点を当てた実践的なガイド。
48,投資リスク管理戦略,堀田直樹,金融情報出版,4200,2020,効果的な投資リスク管理戦略とポートフォリオの構築手法を解説。
49,セキュアなソフトウェア開発,大久保美和,技術書典,5500,2021,セキュアなソフトウェアの設計と実装に関する基本原則と手法を学ぶ。
50,ビジネスインテリジェンス基礎,小川美香,データ出版社,4700,2020,ビジネスインテリジェンスの基本的な原則とデータ分析手法を解説。
51,デジタルマーケティング入門,石田健太,ビジネスブックス,4000,2022,デジタル時代におけるマーケティング戦略と実践的なデジタル広告手法に焦点を当てた入門書。
52,モバイルセキュリティベストプラクティス,中山さやか,技術書典,4400,2022,モバイルアプリケーションのセキュリティ設計と対策手法を解説。
53,ビジネスプランの作成法,渡辺徹,ビジネスパートナーズ,3800,2019,効果的なビジネスプランの作成と実行に向けた手法を学ぶ。
54,クラウドネイティブアーキテクチャ,吉田太郎,技術出版株式会社,5200,2022,クラウドネイティブなアプリケーションの設計と構築に関する包括的なガイド。
55,組織学習の手法,石井花子,ビジネスブックス,4300,2021,組織が持続的に学習し成長するための手法と実践例を解説。
56,JavaScriptデザインパターン,小林次郎,技術書典,4600,2020,JavaScriptにおけるデザインパターンと実践的なプログラミング手法を学ぶ。
57,デジタルトランスフォーメーション戦略,鈴木美和,ビジネスパートナーズ,5400,2019,組織全体でのデジタルトランスフォーメーション戦略の策定と実行に関するガイド。
58,ネットワークプログラミング実践,田村健太,技術書典,4800,2020,ネットワークプログラミングの基本概念から、実践的なアプリケーション開発までを解説。
59,マーケットセグメンテーション,山本直樹,ビジネスブックス,4000,2021,効果的なマーケットセグメンテーションの手法と市場分析に焦点を当てた入門書。
60,サーバレスアーキテクチャ設計,田中美香,技術出版株式会社,5000,2018,サーバレスなアプリケーションの設計とデプロイに関する手法を解説。
61,人材育成プログラム,高橋直樹,ビジネスパートナーズ,3700,2019,組織内での効果的な人材育成プログラムの構築と実践手法を学ぶ。
62,Python機械学習実践ガイド,西村美和,技術書典,5300,2021,Pythonを使用した機械学習の基本理論から実践的な応用までを解説。
63,金融市場データ分析,堀田太郎,データ出版社,4600,2022,金融市場データの分析手法と取引戦略に焦点を当てた実践ガイド。
64,サイバーセキュリティリーダーシップ,矢田美香,技術出版株式会社,4800,2019,組織全体でのサイバーセキュリティリーダーシップの原則と実践手法を解説。
65,効果的なマーケティングコミュニケーション,小野次郎,ビジネスブックス,4200,2020,ターゲットに向けた効果的なマーケティングコミュニケーション戦略を解説。
66,データエンジニアリング入門,松井あや,技術書典,4900,2021,データエンジニアリングの基本概念と実践的なデータ処理手法を学ぶ。
67,戦略的イノベーションマネジメント,大木徹,ビジネスパートナーズ,5500,2020,企業における戦略的なイノベーションマネジメントの手法と実践例を解説。
68,ソフトウェアアーキテクチャ設計,藤田健一,技術出版株式会社,5100,2019,ソフトウェアアーキテクチャの基本原則と設計手法に焦点を当てた実践的なガイド。
69,投資ポートフォリオ最適化,石井さやか,金融情報出版,4300,2020,リスクとリターンのバランスを考慮した投資ポートフォリオ最適化の手法を解説。
70,マイクロサービスアーキテクチャ実践,森下太郎,技術出版株式会社,5200,2022,マイクロサービスアーキテクチャの実践的な設計と運用に関するガイド。
71,戦略的リーダーシップ,大橋美和,ビジネスブックス,4500,2021,組織戦略とリーダーシップの統合に焦点を当てた戦略的リーダーシップのガイド。
72,データウェアハウス最適化,田口健太,技術書典,4700,2020,データウェアハウスの最適化手法とパフォーマンスチューニングに関する実践的な知識。
73,プログラマのためのアルゴリズム,小山直樹,技術出版株式会社,3800,2019,プログラマ向けの基本的なアルゴリズムとデータ構造に焦点を当てた解説書。
74,デジタル戦略の構築,鈴木花子,ビジネスパートナーズ,5100,2019,デジタル時代におけるビジネス戦略の構築手法と実践例を解説。
75,ネットワークセキュリティベストプラクティス,高田徹,技術書典,4400,2022,ネットワークセキュリティのベストプラクティスと脅威対策に関するガイド。
76,人材評価と報酬戦略,小林美和,ビジネスブックス,4300,2021,組織内での効果的な人材評価と報酬戦略の構築手法を学ぶ。
77,リアクティブプログラミング入門,西野健太,技術出版株式会社,4800,2020,リアクティブなアプリケーション開発の基本理論と実践手法を解説。
78,データサイエンスと医療,大谷花子,データ出版社,5400,2021,医療データの解析とデータサイエンスの応用に焦点を当てた入門書。
79,クラウドアーキテクト設計,佐々木徹,技術書典,5000,2022,クラウドシステムのアーキテクト設計とベストプラクティスに関するガイド。
80,ビジネスエシックス,中川美和,ビジネスパートナーズ,3700,2020,ビジネス環境における倫理とエシックスの基本原則と実践的なガイド。
81,機械学習モデル解釈,高木健太,技術書典,4600,2019,機械学習モデルの解釈手法とモデルの透明性向上に焦点を当てた実践書。
82,セキュアなWeb開発,石崎さやか,技術出版株式会社,4300,2021,セキュアなWebアプリケーションの開発手法とセキュリティ対策に関するガイド。
83,デジタルマーケティング戦略,小野徹,ビジネスブックス,4900,2020,デジタルマーケティング戦略の立案と実行に関する実践的な手法を解説。
84,サーバーサイド開発入門,伊東美香,技術書典,4100,2022,サーバーサイド開発の基本概念から、実践的なアプリケーション開発までを解説。
85,データアーキテクチャ設計,松永直樹,データ出版社,5200,2019,データアーキテクチャの設計と実践的なデータ管理手法を学ぶ。
86,クラウドセキュリティアーキテクチャ,田辺美香,技術出版株式会社,5300,2022,クラウド環境におけるセキュリティアーキテクチャの設計と実践手法を解説。
87,リーダーシップ開発プログラム,小野さやか,ビジネスブックス,4100,2021,リーダーシップスキルの開発とチームビルディングの手法を学ぶ。
88,自然言語処理とテキストマイニング,村上健太,技術書典,4700,2020,自然言語処理とテキストマイニングの基本原則と実践手法を学ぶ。
89,デジタル戦略の実践,渡辺花子,ビジネスパートナーズ,5500,2019,デジタル時代におけるビジネス戦略の実践的な構築と実行に関するガイド。
90,フロントエンド開発ベストプラクティス,石井美和,技術出版株式会社,4800,2020,モダンなフロントエンド開発のベストプラクティスと設計原則に焦点を当てた実践的なガイド。
91,イノベーションマネジメント,佐々木太郎,ビジネスブックス,4300,2021,組織内での効果的なイノベーションマネジメントの手法と実践例を解説。
92,Pythonデータサイエンティスト養成,矢田さやか,技術出版株式会社,5100,2018,Pythonを使用したデータサイエンスの基本から実践までの養成ガイド。
93,金融市場リスク管理,山口健太,データ出版社,4900,2019,金融市場におけるリスク管理の基本原則と実践手法を解説。
94,マーケティングデータ分析,大塚美香,金融情報出版,4200,2020,効果的なマーケティングデータ分析の手法と意思決定への応用に焦点を当てた実践ガイド。
95,サイバーセキュリティリスクマネジメント,小川さやか,技術書典,4600,2022,サイバーセキュリティリスクの評価とマネジメントの手法に焦点を当てたガイド。
96,戦略的マーケティングプラン,松本美和,ビジネスパートナーズ,4700,2021,組織のビジョンと目標に基づく戦略的なマーケティングプランの構築手法を解説。
97,データサイエンスとビッグデータ,石田美香,技術書典,5300,2020,データサイエンスとビッグデータの基本原則から実践までの手法を学ぶ。
98,アジャイル開発プラクティス,吉田花子,技術出版株式会社,4000,2021,アジャイルなソフトウェア開発のプラクティスとスクラムの導入手法を解説。
99,サプライチェーンマネジメント,中村健太,データ出版社,5200,2022,効果的なサプライチェーンマネジメントの原則と実践手法を解説。
100,量子コンピュータ応用,田島美和,技術書典,4900,2020,量子コンピュータの応用領域と実践的な利用事例に焦点を当てたガイド。
101,エンタープライズアーキテクチャ,小林花子,ビジネスパートナーズ,4400,2019,エンタープライズ全体のアーキテクチャ設計と最適化手法に関する実践ガイド。
102,プロジェクトポートフォリオマネジメント,松井直樹,技術出版株式会社,5000,2020,プロジェクトポートフォリオの戦略的なマネジメント手法と実践例を解説。
103,マネジメント情報システム,大木美和,データブックス,4600,2021,組織内でのマネジメント情報システムの設計と運用に焦点を当てたガイド。

Writing books.csv


In [ ]:
!head books.csv

ID,title,author,publisher,price,year,description
001,ネットワークセキュリティ入門,山田太郎,技術書典,4500,2021,ネットワークセキュリティの基本原則と実践的な手法に焦点を当てた入門書。
002,ビジネスデータ分析の基礎,田中花子,ビジネスブックス,3800,2020,ビジネスにおけるデータ分析の基本的な手法とツールの使い方を解説。
003,クラウドアーキテクチャデザイン,鈴木次郎,技術出版株式会社,5500,2022,クラウドネイティブなアプリケーションのアーキテクチャデザインとベストプラクティスを詳細に解説。
004,金融取引のアルゴリズム,林美香,データ出版社,5200,2019,金融市場におけるアルゴリズムトレーディングの基本原則と実践手法に焦点を当てた実践的なガイド。
005,人事マネジメントの実践,佐藤健太,ビジネスパートナーズ,4800,2020,組織内の人材マネジメントに関する基本的な原則と実践的な手法を紹介。
006,自然言語処理入門,伊藤さやか,技術書典,4200,2021,自然言語処理の基本概念から、実践的なテキストデータ処理手法までを解説。
007,マーケティング戦略の立案,山口直樹,ビジネスブックス,3600,2018,市場分析、セグメンテーション、ポジショニングなど、マーケティング戦略の構築手法を学ぶ。
008,ディープラーニングアプリケーション,田村徹,技術出版株式会社,6500,2019,ディープラーニングを使用した実践的なアプリケーション開発手法に焦点を当てた実践書。
009,投資ポートフォリオ管理,高橋優一,金融情報出版,3700,2020,効果的な投資ポートフォリオの構築と管理に関する手法を解説。


In [ ]:
import pandas as pd

df = pd.read_csv("books.csv")
df

,ID,title,author,publisher,price,year,description
0,1,ネットワークセキュリティ入門,山田太郎,技術書典,4500,2021,ネットワークセキュリティの基本原則と実践的な手法に焦点を当てた入門書。
1,2,ビジネスデータ分析の基礎,田中花子,ビジネスブックス,3800,2020,ビジネスにおけるデータ分析の基本的な手法とツールの使い方を解説。
2,3,クラウドアーキテクチャデザイン,鈴木次郎,技術出版株式会社,5500,2022,クラウドネイティブなアプリケーションのアーキテクチャデザインとベストプラクティスを詳細に解説。
3,4,金融取引のアルゴリズム,林美香,データ出版社,5200,2019,金融市場におけるアルゴリズムトレーディングの基本原則と実践手法に焦点を当てた実践的なガイド。
4,5,人事マネジメントの実践,佐藤健太,ビジネスパートナーズ,4800,2020,組織内の人材マネジメントに関する基本的な原則と実践的な手法を紹介。
...,...,...,...,...,...,...,...
98,99,サプライチェーンマネジメント,中村健太,データ出版社,5200,2022,効果的なサプライチェーンマネジメントの原則と実践手法を解説。
99,100,量子コンピュータ応用,田島美和,技術書典,4900,2020,量子コンピュータの応用領域と実践的な利用事例に焦点を当てたガイド。
100,101,エンタープライズアーキテクチャ,小林花子,ビジネスパートナーズ,4400,2019,エンタープライズ全体のアーキテクチャ設計と最適化手法に関する実践ガイド。
101,102,プロジェクトポートフォリオマネジメント,松井直樹,技術出版株式会社,5000,2020,プロジェクトポートフォリオの戦略的なマネジメント手法と実践例を解説。


## 類似検索（Similarity Search）の実行

### ターゲットカラムの選択


In [ ]:
descriptions = df["description"]

In [ ]:
type(descriptions)

pandas.core.series.Series

In [ ]:
targets = descriptions.tolist()
type(targets)

list

### LangChain 準備

In [ ]:
import langchain
from langchain.agents.agent_toolkits import create_retriever_tool, create_conversational_retrieval_agent
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.callbacks import StreamlitCallbackHandler
from langchain.schema import BaseRetriever, Document, SystemMessage

In [ ]:
from langchain.globals import set_debug, set_verbose

set_debug(True)
set_verbose(False)

## エンベディングモデルの選択

OpenAIのエンベディングモデルではなく、（連続）利用制限のない、オープンなエンベディングモデルを利用します。

In [1]:
!pip install sentence_transformers==2.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.6 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=01e4d219b8bdaa667e6cd43181e065d26c4f671de3e1045df0a490a81feedc26
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [3]:
from langchain.embeddings import HuggingFaceEmbeddings

text = "テスト文字列"

embeddings = HuggingFaceEmbeddings()
query_result = embeddings.embed_query(text)
doc_result = embeddings.embed_documents([text])

print(len(query_result))
print(query_result)

print(len(doc_result[0]))
print(doc_result)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

768
[0.02339586801826954, -0.051528092473745346, -0.01799503155052662, 0.010700330138206482, 0.039436064660549164, 0.027511082589626312, -0.03647975996136665, 0.06465698033571243, 0.013887857086956501, 0.018204664811491966, 0.03117774985730648, 0.015076836571097374, 0.07378007471561432, 0.029201000928878784, -0.043200548738241196, -0.07707522064447403, -0.0067560975439846516, 0.0345374271273613, 0.045760996639728546, -0.02631659060716629, 0.015634726732969284, 0.012039588764309883, 0.015451109036803246, -0.014117742888629436, 0.015340771526098251, -0.044638700783252716, 0.012640316039323807, -0.012221626937389374, 0.029994942247867584, 0.0002732427674345672, 0.047916773706674576, -0.022249307483434677, -0.0039941188879311085, 0.008768660947680473, 1.510015181338531e-06, -0.017982913181185722, -0.021689971908926964, -0.026938535273075104, -0.018404435366392136, -0.04279183968901634, 0.041685618460178375, -0.0442582331597805, -0.006897759158164263, 0.016141779720783234, 0.027121547609567

In [ ]:
haggingface_embedding_params = embeddings.embed_documents(targets)

In [ ]:
print(len(haggingface_embedding_params[0]))

768


In [ ]:
print(len(targets))
print(len(haggingface_embedding_params))

103
103


In [ ]:
df_vec_added = df.assign(sem_vec=haggingface_embedding_params)
df_vec_added

,ID,title,author,publisher,price,year,description,sem_vec
0,1,ネットワークセキュリティ入門,山田太郎,技術書典,4500,2021,ネットワークセキュリティの基本原則と実践的な手法に焦点を当てた入門書。,"[0.024106020107865334, 0.002243891591206193, -..."
1,2,ビジネスデータ分析の基礎,田中花子,ビジネスブックス,3800,2020,ビジネスにおけるデータ分析の基本的な手法とツールの使い方を解説。,"[0.019792361184954643, -0.0321882963180542, -0..."
2,3,クラウドアーキテクチャデザイン,鈴木次郎,技術出版株式会社,5500,2022,クラウドネイティブなアプリケーションのアーキテクチャデザインとベストプラクティスを詳細に解説。,"[0.02505706250667572, 0.007935326546430588, 0...."
3,4,金融取引のアルゴリズム,林美香,データ出版社,5200,2019,金融市場におけるアルゴリズムトレーディングの基本原則と実践手法に焦点を当てた実践的なガイド。,"[0.004268913064152002, 0.00947908777743578, -0..."
4,5,人事マネジメントの実践,佐藤健太,ビジネスパートナーズ,4800,2020,組織内の人材マネジメントに関する基本的な原則と実践的な手法を紹介。,"[0.0342194028198719, -0.004347567446529865, -0..."
...,...,...,...,...,...,...,...,...
98,99,サプライチェーンマネジメント,中村健太,データ出版社,5200,2022,効果的なサプライチェーンマネジメントの原則と実践手法を解説。,"[0.031015705317258835, -0.028873931616544724, ..."
99,100,量子コンピュータ応用,田島美和,技術書典,4900,2020,量子コンピュータの応用領域と実践的な利用事例に焦点を当てたガイド。,"[0.012202777899801731, -0.01924457773566246, -..."
100,101,エンタープライズアーキテクチャ,小林花子,ビジネスパートナーズ,4400,2019,エンタープライズ全体のアーキテクチャ設計と最適化手法に関する実践ガイド。,"[0.03868181258440018, -0.01839403621852398, -0..."
101,102,プロジェクトポートフォリオマネジメント,松井直樹,技術出版株式会社,5000,2020,プロジェクトポートフォリオの戦略的なマネジメント手法と実践例を解説。,"[0.05236094072461128, -0.05124589428305626, -0..."


In [ ]:
docs = []
#for entry in philo_dataset:
for index, row in df_vec_added.iterrows():
  doc = {
      "_id": row['ID'],
      "text": row['description'],
      "$vector": row['sem_vec'],
    }
  docs.append(doc)

In [ ]:
for doc in docs:
  print(doc)

In [ ]:
collection = db.create_collection("vector_test", dimension=len(haggingface_embedding_params[0]), metric="cosine")

In [ ]:
for doc in docs:
  res = collection.upsert(doc)

In [ ]:
txt = 'データサイエンスについての書籍'

emb_query = embeddings.embed_query(txt)
len(emb_query)

768

In [ ]:
results = collection.vector_find(emb_query, limit=5, fields={"text", "$vector"})

for document in results:
    print(document)

{'_id': 78, 'text': '医療データの解析とデータサイエンスの応用に焦点を当てた入門書。', '$vector': [0.034044794738292694, -0.017598794773221016, -0.03157713636755943, 0.045738816261291504, 0.022502297535538673, 0.004622526466846466, -0.0373268723487854, 0.040677063167095184, -0.01582375355064869, -0.01635187864303589, -0.012025615200400352, 0.01967018097639084, 0.055693645030260086, 0.0023237804416567087, -0.031024910509586334, -0.060869842767715454, -0.013411606661975384, 0.01149777602404356, -0.007272401824593544, -0.013135017827153206, 0.02718123234808445, -0.008689021691679955, 0.014446353539824486, -0.04476960375905037, 0.04020700231194496, -0.0471651628613472, 0.021251147612929344, -0.03416278585791588, 0.04725804179906845, -0.022962037473917007, 0.08957129716873169, -0.010219097137451172, 0.004555325955152512, -0.025082148611545563, 2.0055874756508274e-06, -0.011692995205521584, -0.02966170385479927, 0.00758295739069581, 0.011029058136045933, -0.055672697722911835, -0.08335306495428085, 0.02266176976263523, 0.0

In [ ]:
txt = '量子コンピュータ'

emb_query = embeddings.embed_query(txt)
len(emb_query)

results = collection.vector_find(emb_query, limit=5, fields={"text", "$vector"})

for document in results:
    print(document)

{'_id': 36, 'text': '量子コンピューティングの基礎理論から実践的な応用までを解説。', '$vector': [0.009070678614079952, -0.04001716151833534, -0.006851665675640106, 0.024777932092547417, 0.016087645664811134, 0.005538738798350096, -0.049673665314912796, 0.054408807307481766, 0.02979297935962677, -0.00292282085865736, -0.03147730976343155, 0.01930518075823784, 0.03445550426840782, -0.019276749342679977, -0.039170920848846436, -0.06207604333758354, 0.003078525187447667, 0.0630868598818779, 0.0431707464158535, 0.0020192081574350595, 0.050345998257398605, -0.02170218899846077, 0.03463416174054146, 0.030700266361236572, 0.013894462957978249, -0.050135839730501175, 0.04077034443616867, -0.030260324478149414, 0.022563235834240913, -0.014012530446052551, 0.05012514814734459, -0.039426036179065704, -0.011944976635277271, -0.05201958119869232, 1.860235329331772e-06, -0.02258562110364437, -0.024801678955554962, -0.015126039274036884, 0.03801695257425308, -0.08303191512823105, -0.03780224174261093, -0.0974254161119461, -0.025916

## LangChainアプリケーションの構築

### OpenAI

In [ ]:
import os
import openai
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass()
#openai.api_key = os.getenv("OPENAI_API_KEY")

··········


In [ ]:
import openai
from openai import OpenAI

client = OpenAI()
client.models.list()

SyncPage[Model](data=[Model(id='curie-search-query', created=1651172509, object='model', owned_by='openai-dev'), Model(id='babbage-search-query', created=1651172509, object='model', owned_by='openai-dev'), Model(id='dall-e-3', created=1698785189, object='model', owned_by='system'), Model(id='babbage-search-document', created=1651172510, object='model', owned_by='openai-dev'), Model(id='dall-e-2', created=1698798177, object='model', owned_by='system'), Model(id='gpt-4-1106-preview', created=1698957206, object='model', owned_by='system'), Model(id='gpt-3.5-turbo-0301', created=1677649963, object='model', owned_by='openai'), Model(id='text-embedding-ada-002', created=1671217299, object='model', owned_by='openai-internal'), Model(id='davinci-search-query', created=1651172505, object='model', owned_by='openai-dev'), Model(id='tts-1-hd-1106', created=1699053533, object='model', owned_by='system'), Model(id='gpt-4-0613', created=1686588896, object='model', owned_by='openai'), Model(id='tts-1-

In [ ]:
from langchain_community.vectorstores import AstraDB
vstore = AstraDB(
    embedding=embeddings,
    collection_name="book_collection",
    token=YOUR_TOKEN,
    api_endpoint=API_ENDPOINT,
)

以下のテーブルが作成される。

```
CREATE TABLE default_keyspace.book_collection (
    key frozen<tuple<tinyint, text>> PRIMARY KEY,
    array_contains set<text>,
    array_size map<text, int>,
    doc_json text,
    exist_keys set<text>,
    query_bool_values map<text, tinyint>,
    query_dbl_values map<text, decimal>,
    query_null_values set<text>,
    query_text_values map<text, text>,
    query_timestamp_values map<text, timestamp>,
    query_vector_value vector<float, 768>,
    tx_id timeuuid
) WITH additional_write_policy = '99p'
    AND bloom_filter_fp_chance = 0.01
    AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}
    AND comment = ''
    AND compaction = {'class': 'org.apache.cassandra.db.compaction.UnifiedCompactionStrategy'}
    AND compression = {'chunk_length_in_kb': '16', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}
    AND crc_check_chance = 1.0
    AND default_time_to_live = 0
    AND gc_grace_seconds = 864000
    AND max_index_interval = 2048
    AND memtable_flush_period_in_ms = 0
    AND min_index_interval = 128
    AND read_repair = 'BLOCKING'
    AND speculative_retry = '99p';
CREATE CUSTOM INDEX book_collection_array_contains ON default_keyspace.book_collection (values(array_contains)) USING 'StorageAttachedIndex';
CREATE CUSTOM INDEX book_collection_array_size ON default_keyspace.book_collection (entries(array_size)) USING 'StorageAttachedIndex';
CREATE CUSTOM INDEX book_collection_exists_keys ON default_keyspace.book_collection (values(exist_keys)) USING 'StorageAttachedIndex';
CREATE CUSTOM INDEX book_collection_query_bool_values ON default_keyspace.book_collection (entries(query_bool_values)) USING 'StorageAttachedIndex';
CREATE CUSTOM INDEX book_collection_query_dbl_values ON default_keyspace.book_collection (entries(query_dbl_values)) USING 'StorageAttachedIndex';
CREATE CUSTOM INDEX book_collection_query_null_values ON default_keyspace.book_collection (values(query_null_values)) USING 'StorageAttachedIndex';
CREATE CUSTOM INDEX book_collection_query_text_values ON default_keyspace.book_collection (entries(query_text_values)) USING 'StorageAttachedIndex';
CREATE CUSTOM INDEX book_collection_query_timestamp_values ON default_keyspace.book_collection (entries(query_timestamp_values)) USING 'StorageAttachedIndex';
CREATE CUSTOM INDEX book_collection_query_vector_value ON default_keyspace.book_collection (query_vector_value) USING 'StorageAttachedIndex' WITH OPTIONS = {'similarity_function': 'cosine'};

```

In [ ]:
df

,ID,title,author,publisher,price,year,description
0,1,ネットワークセキュリティ入門,山田太郎,技術書典,4500,2021,ネットワークセキュリティの基本原則と実践的な手法に焦点を当てた入門書。
1,2,ビジネスデータ分析の基礎,田中花子,ビジネスブックス,3800,2020,ビジネスにおけるデータ分析の基本的な手法とツールの使い方を解説。
2,3,クラウドアーキテクチャデザイン,鈴木次郎,技術出版株式会社,5500,2022,クラウドネイティブなアプリケーションのアーキテクチャデザインとベストプラクティスを詳細に解説。
3,4,金融取引のアルゴリズム,林美香,データ出版社,5200,2019,金融市場におけるアルゴリズムトレーディングの基本原則と実践手法に焦点を当てた実践的なガイド。
4,5,人事マネジメントの実践,佐藤健太,ビジネスパートナーズ,4800,2020,組織内の人材マネジメントに関する基本的な原則と実践的な手法を紹介。
...,...,...,...,...,...,...,...
98,99,サプライチェーンマネジメント,中村健太,データ出版社,5200,2022,効果的なサプライチェーンマネジメントの原則と実践手法を解説。
99,100,量子コンピュータ応用,田島美和,技術書典,4900,2020,量子コンピュータの応用領域と実践的な利用事例に焦点を当てたガイド。
100,101,エンタープライズアーキテクチャ,小林花子,ビジネスパートナーズ,4400,2019,エンタープライズ全体のアーキテクチャ設計と最適化手法に関する実践ガイド。
101,102,プロジェクトポートフォリオマネジメント,松井直樹,技術出版株式会社,5000,2020,プロジェクトポートフォリオの戦略的なマネジメント手法と実践例を解説。


In [ ]:
id_series = df["ID"]
ids = id_series.to_list()
type(ids)

list

In [ ]:
texts = targets
type(texts)

list

In [ ]:
ids

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103]

In [ ]:
df_for_meta = df.drop(['ID','description'], axis=1)
df_for_meta

,title,author,publisher,price,year
0,ネットワークセキュリティ入門,山田太郎,技術書典,4500,2021
1,ビジネスデータ分析の基礎,田中花子,ビジネスブックス,3800,2020
2,クラウドアーキテクチャデザイン,鈴木次郎,技術出版株式会社,5500,2022
3,金融取引のアルゴリズム,林美香,データ出版社,5200,2019
4,人事マネジメントの実践,佐藤健太,ビジネスパートナーズ,4800,2020
...,...,...,...,...,...
98,サプライチェーンマネジメント,中村健太,データ出版社,5200,2022
99,量子コンピュータ応用,田島美和,技術書典,4900,2020
100,エンタープライズアーキテクチャ,小林花子,ビジネスパートナーズ,4400,2019
101,プロジェクトポートフォリオマネジメント,松井直樹,技術出版株式会社,5000,2020


In [ ]:
metadatas = df_for_meta.to_dict(orient='records')
metadatas[0]

{'title': 'ネットワークセキュリティ入門',
 'author': '山田太郎',
 'publisher': '技術書典',
 'price': 4500,
 'year': 2021}

In [ ]:
for cnt, text in enumerate(texts, start=0):
  print(cnt)
  print(text)
  print(metadatas[cnt])

0
ネットワークセキュリティの基本原則と実践的な手法に焦点を当てた入門書。
{'title': 'ネットワークセキュリティ入門', 'author': '山田太郎', 'publisher': '技術書典', 'price': 4500, 'year': 2021}
1
ビジネスにおけるデータ分析の基本的な手法とツールの使い方を解説。
{'title': 'ビジネスデータ分析の基礎', 'author': '田中花子', 'publisher': 'ビジネスブックス', 'price': 3800, 'year': 2020}
2
クラウドネイティブなアプリケーションのアーキテクチャデザインとベストプラクティスを詳細に解説。
{'title': 'クラウドアーキテクチャデザイン', 'author': '鈴木次郎', 'publisher': '技術出版株式会社', 'price': 5500, 'year': 2022}
3
金融市場におけるアルゴリズムトレーディングの基本原則と実践手法に焦点を当てた実践的なガイド。
{'title': '金融取引のアルゴリズム', 'author': '林美香', 'publisher': 'データ出版社', 'price': 5200, 'year': 2019}
4
組織内の人材マネジメントに関する基本的な原則と実践的な手法を紹介。
{'title': '人事マネジメントの実践', 'author': '佐藤健太', 'publisher': 'ビジネスパートナーズ', 'price': 4800, 'year': 2020}
5
自然言語処理の基本概念から、実践的なテキストデータ処理手法までを解説。
{'title': '自然言語処理入門', 'author': '伊藤さやか', 'publisher': '技術書典', 'price': 4200, 'year': 2021}
6
市場分析、セグメンテーション、ポジショニングなど、マーケティング戦略の構築手法を学ぶ。
{'title': 'マーケティング戦略の立案', 'author': '山口直樹', 'publisher': 'ビジネスブックス', 'price': 3600, 'year': 2018}
7
ディープラーニングを使用した実践的なアプリケーション開発手

In [ ]:
docs = []
for cnt, text in enumerate(texts, start=0):
  metadata = metadatas[cnt]
  doc = Document(page_content=text, metadata=metadata)
  docs.append(doc)

In [ ]:
docs

[Document(page_content='ネットワークセキュリティの基本原則と実践的な手法に焦点を当てた入門書。', metadata={'title': 'ネットワークセキュリティ入門', 'author': '山田太郎', 'publisher': '技術書典', 'price': 4500, 'year': 2021}),
 Document(page_content='ビジネスにおけるデータ分析の基本的な手法とツールの使い方を解説。', metadata={'title': 'ビジネスデータ分析の基礎', 'author': '田中花子', 'publisher': 'ビジネスブックス', 'price': 3800, 'year': 2020}),
 Document(page_content='クラウドネイティブなアプリケーションのアーキテクチャデザインとベストプラクティスを詳細に解説。', metadata={'title': 'クラウドアーキテクチャデザイン', 'author': '鈴木次郎', 'publisher': '技術出版株式会社', 'price': 5500, 'year': 2022}),
 Document(page_content='金融市場におけるアルゴリズムトレーディングの基本原則と実践手法に焦点を当てた実践的なガイド。', metadata={'title': '金融取引のアルゴリズム', 'author': '林美香', 'publisher': 'データ出版社', 'price': 5200, 'year': 2019}),
 Document(page_content='組織内の人材マネジメントに関する基本的な原則と実践的な手法を紹介。', metadata={'title': '人事マネジメントの実践', 'author': '佐藤健太', 'publisher': 'ビジネスパートナーズ', 'price': 4800, 'year': 2020}),
 Document(page_content='自然言語処理の基本概念から、実践的なテキストデータ処理手法までを解説。', metadata={'title': '自然言語処理入門', 'author': '伊藤さやか', 'publisher': '技術書

In [ ]:
inserted_ids = vstore.add_documents(docs)
print(f"\nInserted {len(inserted_ids)} documents.")


Inserted 103 documents.


In [ ]:
retriever_from_vectorstore = vstore.as_retriever()

In [ ]:
def create_chatbot_with_retriever(lang: str, retriever):
    print(f"Creating chatbot for {lang}...")
    llm = ChatOpenAI(temperature=0, streaming=True)

    retriever_tool = create_retriever_tool(
        retriever, "book_retrevier", "Useful when searching for books from a book store. Prices are in YEN.")
    system_message = """
    You are a customer service of a book store and you are asked to pick books for a customer.
    You must try to find books related to given questions first.
    You must use the books_retreiver.
    You must not provide any information other than books that you get from books_retriever.
    You should behave as a bookstore clerk.
    """
    if lang == "Japanese":
        system_message = f"{system_message} All the responses should be in Japanese language."
    message = SystemMessage(content=system_message)
    agent_executor = create_conversational_retrieval_agent(
        llm=llm, tools=[retriever_tool], system_message=message, verbose=True)
    return agent_executor

In [ ]:
lang = "Japanese"
chatbot = create_chatbot_with_retriever(lang, retriever_from_vectorstore)

Creating chatbot for Japanese...


In [ ]:
prompt = "機械学習についての本を教えて"
vstore.similarity_search(prompt)

[Document(page_content='データサイエンスにおける統計学の基本と高度な手法までを解説。', metadata={'title': 'データサイエンティストのための統計学', 'author': '小林太郎', 'publisher': 'データ書典', 'price': 4400, 'year': 2020}),
 Document(page_content='データサイエンスとビジネスアナリティクスの基本概念と手法を学ぶ。', metadata={'title': 'データサイエンスとビジネス分析', 'author': '鈴木健一', 'publisher': '金融情報出版', 'price': 3600, 'year': 2020}),
 Document(page_content='セキュアなソフトウェアの設計と実装に関する基本原則と手法を学ぶ。', metadata={'title': 'セキュアなソフトウェア開発', 'author': '大久保美和', 'publisher': '技術書典', 'price': 5500, 'year': 2021}),
 Document(page_content='データサイエンスとビッグデータの基本原則から実践までの手法を学ぶ。', metadata={'title': 'データサイエンスとビッグデータ', 'author': '石田美香', 'publisher': '技術書典', 'price': 5300, 'year': 2020})]

In [ ]:
prompt = "機械学習についての本を教えて"
result = chatbot.invoke({
            "input": prompt})
print(result["output"])

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "機械学習についての本を教えて",
  "chat_history": []
}
[llm/start] [1:chain:AgentExecutor > 2:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: \n    You are a customer service of a book store and you are asked to pick books for a customer.\n    You must try to find books related to given questions first.\n    You must use the books_retreiver.\n    You must not provide any information other than books that you get from books_retriever.\n    You should behave as a bookstore clerk.\n     All the responses should be in Japanese language.\nHuman: 機械学習についての本を教えて"
  ]
}
[llm/end] [1:chain:AgentExecutor > 2:llm:ChatOpenAI] [2.68s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "",
        "generation_info": {
          "finish_reason": "function_call"
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
 

## 付録. 様々な検索メソッド

In [ ]:
vstore.similarity_search_with_relevance_scores(prompt)

[(Document(page_content='データサイエンスにおける統計学の基本と高度な手法までを解説。', metadata={'title': 'データサイエンティストのための統計学', 'author': '小林太郎', 'publisher': 'データ書典', 'price': 4400, 'year': 2020}),
  0.8979914),
 (Document(page_content='データサイエンスとビジネスアナリティクスの基本概念と手法を学ぶ。', metadata={'title': 'データサイエンスとビジネス分析', 'author': '鈴木健一', 'publisher': '金融情報出版', 'price': 3600, 'year': 2020}),
  0.8778603),
 (Document(page_content='セキュアなソフトウェアの設計と実装に関する基本原則と手法を学ぶ。', metadata={'title': 'セキュアなソフトウェア開発', 'author': '大久保美和', 'publisher': '技術書典', 'price': 5500, 'year': 2021}),
  0.87310207),
 (Document(page_content='データサイエンスとビッグデータの基本原則から実践までの手法を学ぶ。', metadata={'title': 'データサイエンスとビッグデータ', 'author': '石田美香', 'publisher': '技術書典', 'price': 5300, 'year': 2020}),
  0.8663833)]

In [ ]:
vstore.similarity_search_with_relevance_scores(prompt,filter={'publisher':'技術書典'})

[(Document(page_content='セキュアなソフトウェアの設計と実装に関する基本原則と手法を学ぶ。', metadata={'title': 'セキュアなソフトウェア開発', 'author': '大久保美和', 'publisher': '技術書典', 'price': 5500, 'year': 2021}),
  0.87310207),
 (Document(page_content='データサイエンスとビッグデータの基本原則から実践までの手法を学ぶ。', metadata={'title': 'データサイエンスとビッグデータ', 'author': '石田美香', 'publisher': '技術書典', 'price': 5300, 'year': 2020}),
  0.8663833),
 (Document(page_content='データエンジニアリングの基本概念と実践的なデータ処理手法を学ぶ。', metadata={'title': 'データエンジニアリング入門', 'author': '松井あや', 'publisher': '技術書典', 'price': 4900, 'year': 2021}),
  0.8509538),
 (Document(page_content='自然言語処理とテキストマイニングの基本原則と実践手法を学ぶ。', metadata={'title': '自然言語処理とテキストマイニング', 'author': '村上健太', 'publisher': '技術書典', 'price': 4700, 'year': 2020}),
  0.8500574)]

In [ ]:
vstore.similarity_search_with_relevance_scores(prompt,filter={'author':'西村美和'})

[(Document(page_content='Pythonを使用した機械学習の基本理論から実践的な応用までを解説。', metadata={'title': 'Python機械学習実践ガイド', 'author': '西村美和', 'publisher': '技術書典', 'price': 5300, 'year': 2021}),
  0.7992706)]

In [ ]:
vstore.similarity_search_with_relevance_scores(prompt,filter={'publisher':'技術書典', 'author':'西村美和'})

[(Document(page_content='Pythonを使用した機械学習の基本理論から実践的な応用までを解説。', metadata={'title': 'Python機械学習実践ガイド', 'author': '西村美和', 'publisher': '技術書典', 'price': 5300, 'year': 2021}),
  0.7992706)]

In [ ]:
vstore.similarity_search_with_relevance_scores(prompt,filter={'publisher':'技術書典', 'title':'自然言語'})

[]

In [ ]:
vstore.max_marginal_relevance_search(prompt)

[Document(page_content='データサイエンスにおける統計学の基本と高度な手法までを解説。', metadata={'title': 'データサイエンティストのための統計学', 'author': '小林太郎', 'publisher': 'データ書典', 'price': 4400, 'year': 2020}),
 Document(page_content='自然言語処理と機械翻訳の基本原則と応用手法を学ぶ。', metadata={'title': '自然言語処理と機械翻訳', 'author': '山岸直樹', 'publisher': '技術出版株式会社', 'price': 5300, 'year': 2018}),
 Document(page_content='医療データの解析とデータサイエンスの応用に焦点を当てた入門書。', metadata={'title': 'データサイエンスと医療', 'author': '大谷花子', 'publisher': 'データ出版社', 'price': 5400, 'year': 2021}),
 Document(page_content='ビジネス環境における倫理とエシックスの基本原則と実践的なガイド。', metadata={'title': 'ビジネスエシックス', 'author': '中川美和', 'publisher': 'ビジネスパートナーズ', 'price': 3700, 'year': 2020})]

In [ ]:
vstore.max_marginal_relevance_search(prompt, lambda_mult=0.1)

[Document(page_content='データサイエンスにおける統計学の基本と高度な手法までを解説。', metadata={'title': 'データサイエンティストのための統計学', 'author': '小林太郎', 'publisher': 'データ書典', 'price': 4400, 'year': 2020}),
 Document(page_content='自然言語処理と機械翻訳の基本原則と応用手法を学ぶ。', metadata={'title': '自然言語処理と機械翻訳', 'author': '山岸直樹', 'publisher': '技術出版株式会社', 'price': 5300, 'year': 2018}),
 Document(page_content='ビジネス環境における倫理とエシックスの基本原則と実践的なガイド。', metadata={'title': 'ビジネスエシックス', 'author': '中川美和', 'publisher': 'ビジネスパートナーズ', 'price': 3700, 'year': 2020}),
 Document(page_content='リーダーシップと心理学の関連性と、組織内での実践手法を解説。', metadata={'title': 'リーダーシップ心理学', 'author': '大西美香', 'publisher': 'ビジネスパートナーズ', 'price': 4100, 'year': 2018})]

In [ ]:
vstore.max_marginal_relevance_search(prompt, lambda_mult=0.9)

[Document(page_content='データサイエンスにおける統計学の基本と高度な手法までを解説。', metadata={'title': 'データサイエンティストのための統計学', 'author': '小林太郎', 'publisher': 'データ書典', 'price': 4400, 'year': 2020}),
 Document(page_content='データサイエンスとビジネスアナリティクスの基本概念と手法を学ぶ。', metadata={'title': 'データサイエンスとビジネス分析', 'author': '鈴木健一', 'publisher': '金融情報出版', 'price': 3600, 'year': 2020}),
 Document(page_content='セキュアなソフトウェアの設計と実装に関する基本原則と手法を学ぶ。', metadata={'title': 'セキュアなソフトウェア開発', 'author': '大久保美和', 'publisher': '技術書典', 'price': 5500, 'year': 2021}),
 Document(page_content='組織が持続的に学習し成長するための手法と実践例を解説。', metadata={'title': '組織学習の手法', 'author': '石井花子', 'publisher': 'ビジネスブックス', 'price': 4300, 'year': 2021})]

In [ ]:
# Retrieve more documents with higher diversity
# Useful if your dataset has many similar documents
retriever_a = vstore.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 6, 'lambda_mult': 0.25}
)

# Fetch more documents for the MMR algorithm to consider
# But only return the top 5
retriever_b = vstore.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 5, 'fetch_k': 50}
)

# Only retrieve documents that have a relevance score
# Above a certain threshold
retriever_c = vstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={'score_threshold': 0.95}
)

# Only get the single most similar document from the dataset
retriever_d = vstore.as_retriever(search_kwargs={'k': 1})

# Use a filter to only retrieve documents from a specific paper
retriever_e = vstore.as_retriever(
    search_kwargs={'filter': {'paper_title':'GPT-4 Technical Report'}}
)